In [22]:
import pandas as pd
import json

In [23]:
csvpath='purchase_data.json'
with open('purchase_data.json') as data_file:
    data = json.load(data_file)
game_file=pd.DataFrame(data)
game_file.head(10)


,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59
5,20,Male,10,Sleepwalker,1.73,Tanimnya91
6,20,Male,153,Mercenary Sabre,4.57,Undjaskla97
7,29,Female,169,"Interrogator, Blood Blade of the Queen",3.32,Iathenudil29
8,25,Male,118,"Ghost Reaver, Longsword of Magic",2.77,Sondenasta63
9,31,Male,99,"Expiration, Warscythe Of Lost Worlds",4.53,Hilaerin92


In [24]:
game_file.count()

Age          780
Gender       780
Item ID      780
Item Name    780
Price        780
SN           780
dtype: int64

In [25]:
game_file.dtypes

Age            int64
Gender        object
Item ID        int64
Item Name     object
Price        float64
SN            object
dtype: object

In [26]:
#Player Count

#dropping the all the rows with duplicates SNs 
game_file_dup_removed=game_file.drop_duplicates(['SN'])
Total=game_file_dup_removed['SN'].count()

Total_Players=pd.DataFrame({"Total Players":[ Total]})
Total_Players


,Total Players
0,573


In [27]:
#Purchasing Analysis (Total)

#unique() returns an array, so used len
x=game_file["Item ID"].unique()
unique_items=len(x)

Average_price=game_file["Price"].mean()
#just counting the no.of rows
Number_purchases=game_file["Price"].count()
Total_Revenue=game_file["Price"].sum()

Purchasing_Analysis_Total=pd.DataFrame({"Number of unique items": [unique_items], "Average Price":[Average_price],"Number of Purchases":
                                 [Number_purchases],"Total Revenue": Total_Revenue})

Purchasing_Analysis_Total["Average Price"] = Purchasing_Analysis_Total["Average Price"].map("${:.2f}".format)
Purchasing_Analysis_Total["Total Revenue"] = Purchasing_Analysis_Total["Total Revenue"].map("${:.2f}".format)
Purchasing_Analysis_Total



,Average Price,Number of Purchases,Number of unique items,Total Revenue
0,$2.93,780,183,$2286.33


In [28]:
#Gender Demographics

#**need to use duplicate removed file
Male_count=game_file_dup_removed.loc[game_file["Gender"]=="Male","Gender"].count()

Female_count=game_file_dup_removed.loc[game_file["Gender"]=="Female","Gender"].count()
Total_count=game_file_dup_removed["Gender"].count()
Other_ND_count=Total_count-(Male_count+Female_count)

Male_percent=round(((Male_count/Total_count)*100),2)
Female_percent=round(((Female_count/Total_count)*100),2)
Other_percent=round(((Other_ND_count/Total_count)*100),2)

Gender_demo=pd.DataFrame({"Gender":["Female","Male","Other / Non-Disclosed"],
                          "Percentage of Players":[Female_percent,Male_percent,Other_percent],
                         "Total Count":[Female_count,Male_count,Other_ND_count]})
#just for the display without the numbered index
Gender_demo_final=Gender_demo.set_index(['Gender'])
Gender_demo_final




,Percentage of Players,Total Count
Gender,,
Female,17.45,100
Male,81.15,465
Other / Non-Disclosed,1.40,8


In [29]:
#Purchasing Analysis (Gender)

#A LANDY ALTERNATE METHOD
#y=game_file.groupby(["Gender"])
#m=y['Price'].mean().head()
#IT RETURNS AN OBJECT ARRAY, SO U CAN USE ARRAY FUNCTIONS
#Male_normalised=n[1]/Male_count
#new=pd.DataFrame({"Gender":"Male","Purchase count":[Male_count],"Average Purchase Price":[m[0],m[1],m[2]],"Normalized Tools": [Male_normalised]})
#Gender_Analysis=new.set_index(['Gender'])


Gender_file=game_file.groupby(['Gender'])
Gender_Purchase_Analysis=pd.DataFrame({"Purchase count": game_file['Gender'].value_counts(),
                                       #=Purchase count": Gender_file['Gender'].count()
                             "Average Purchase Price": Gender_file['Price'].mean().map("${:.2f}".format),
                             "Total Purchase Value":Gender_file['Price'].sum().map("${:.2f}".format),
                             "Normalized Totals":(Gender_file['Price'].sum()/Gender_demo_final['Total Count'])})
                              #the division is like matrix, element by element for the index values. check the num & deno by printing seperately

Gender_Purchase_Analysis["Normalized Totals"] = Gender_Purchase_Analysis["Normalized Totals"].map("${:.2f}".format)
Gender_Purchase_Analysis.index.names = ['Gender']
Gender_Purchase_Analysis




,Average Purchase Price,Normalized Totals,Purchase count,Total Purchase Value
Gender,,,,
Female,$2.82,$3.83,136,$382.91
Male,$2.95,$4.02,633,$1867.68
Other / Non-Disclosed,$3.25,$4.47,11,$35.74


In [30]:
#Age Demographics

bins = [0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 999999]
age_range = ['0-4', '5-8', '9-12', '13-16', '17-20','21-24','25-28','29-32','33-36','37-40','>40']

#here , duplicates need to be considered coz its a age demographics
#game_file_new=game_file.drop_duplicates(['Gender','SN'])--deletes the rows when duplication in both Gender and SN at the same time
dup_copy=game_file_dup_removed
dup_copy["Age Range (in yrs)"] = pd.cut(dup_copy["Age"], bins, labels=age_range)

Age_demo_file=dup_copy.groupby("Age Range (in yrs)")


Age_demo_analysis=pd.DataFrame({"Purchase Count":dup_copy['Age Range (in yrs)'].value_counts(),
                               "Average Purchase Price":Age_demo_file['Price'].mean().map("${:.2f}".format),
                               "Total Purchase Value":Age_demo_file['Price'].sum().map("${:.2f}".format),
                               "Normalized Totals": ((Age_demo_file['Price'].sum())/(dup_copy['Age Range (in yrs)'].value_counts())).map("${:.2f}".format)})
                                #the sequence of age range (index) while division changes but final result considers the change--kind of vlookup

#re indexed and named the age range for better view    
Age_demo_analysis.index.names = ['Age Range (in yrs)']
Age_demo_analysis.reindex(['0-4', '5-8', '9-12', '13-16', '17-20','21-24','25-28','29-32','33-36','37-40','>40'])



C:\Users\karis\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Average Purchase Price,Normalized Totals,Purchase Count,Total Purchase Value
Age Range (in yrs),,,,
0-4,$nan,$nan,0,$0.00
5-8,$2.87,$2.87,13,$37.33
9-12,$3.42,$3.42,18,$61.49
13-16,$2.76,$2.76,64,$176.96
17-20,$2.94,$2.94,120,$352.78
21-24,$2.95,$2.95,186,$548.50
25-28,$3.07,$3.07,75,$230.01
29-32,$3.16,$3.16,44,$139.15
33-36,$3.14,$3.14,28,$87.82


In [31]:
#Top Spenders

spenders_file=game_file.groupby('SN')

#try to avoid value_counts() on groupby files for duplication of a column/index
Top_Spenders_df=pd.DataFrame({"Purchase Count": game_file['SN'].value_counts(),
                            "Average Purchase Price": spenders_file['Price'].mean().map("${:.2f}".format),
                            "Total Purchase Value":spenders_file['Price'].sum().map("${:.2f}".format)})

Top_5_sort=Top_Spenders_df.sort_values("Total Purchase Value", ascending=False)


Top_5_Spenders=Top_5_sort.head(5)
Top_5_Spenders.index.names = ['SN']
Top_5_Spenders

,Average Purchase Price,Purchase Count,Total Purchase Value
SN,,,
Qarwen67,$2.49,4,$9.97
Sondim43,$3.13,3,$9.38
Tillyrin30,$3.06,3,$9.19
Lisistaya47,$3.06,3,$9.19
Tyisriphos58,$4.59,2,$9.18


In [32]:
#Most Popular Items

Item_file=game_file.groupby(['Item ID','Item Name','Price'])

Item_file_df=pd.DataFrame({"Purchase Count": Item_file['Item ID'].count(),
                           #hard to visualize the above. it is counting each row for each Item ID tallying with the group by indices
                          "Total Purchase Value":Item_file['Price'].sum().map("${:.2f}".format)})
                                    
Item_file_sort=Item_file_df.sort_values("Purchase Count",ascending=False)
#resetting the price for '$' formatting
Top_5_Items_bypurchase=Item_file_sort.reset_index('Price').head(5)
Top_5_Items_bypurchase['Price']=Top_5_Items_bypurchase['Price'].map("${:.2f}".format)
Top_5_Items_bypurchase


,,Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
84,Arcane Gem,$2.23,11,$24.53
31,Trickster,$2.07,9,$18.63
175,Woeful Adamantite Claymore,$1.24,9,$11.16
13,Serenity,$1.49,9,$13.41


In [33]:
#Most Profitable Items

Item_file2=game_file.groupby(['Item ID','Item Name','Price'])

Item_file2_df=pd.DataFrame({"Purchase Count": Item_file['Item ID'].count(),
                           #hard to visualize the above. it is counting Item ID tallying with the groupby Item ID.No. of occurences of particular Item ID=count
                          "Total Purchase Value":Item_file['Price'].sum()})
                                       
Item_file2_sort=Item_file2_df.sort_values("Total Purchase Value",ascending=False)
Item_file2_sort['Total Purchase Value']=Item_file2_sort['Total Purchase Value'].map("${:.2f}".format)
Top_5_profitable_items=Item_file2_sort.reset_index('Price').head(5)
Top_5_profitable_items['Price']=Top_5_profitable_items['Price'].map("${:.2f}".format)
Top_5_profitable_items


,,Price,Purchase Count,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,$4.14,9,$37.26
115,Spectral Diamond Doomblade,$4.25,7,$29.75
32,Orenmir,$4.95,6,$29.70
103,Singed Scalpel,$4.87,6,$29.22
107,"Splitter, Foe Of Subtlety",$3.61,8,$28.88
